In [1]:
import numpy as np
import nltk
import underthesea
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer 
import heapq

In [2]:
vi_stopwords  = []
with open("../vietnamese-stopwords.txt", 'r', encoding='utf-8') as f:
    for line in f:
        vi_stopwords.append(line.strip())
    
vi_stopwords

['a lô',
 'a ha',
 'ai',
 'ai ai',
 'ai nấy',
 'ai đó',
 'alô',
 'amen',
 'anh',
 'anh ấy',
 'ba',
 'ba ba',
 'ba bản',
 'ba cùng',
 'ba họ',
 'ba ngày',
 'ba ngôi',
 'ba tăng',
 'bao giờ',
 'bao lâu',
 'bao nhiêu',
 'bao nả',
 'bay biến',
 'biết',
 'biết bao',
 'biết bao nhiêu',
 'biết chắc',
 'biết chừng nào',
 'biết mình',
 'biết mấy',
 'biết thế',
 'biết trước',
 'biết việc',
 'biết đâu',
 'biết đâu chừng',
 'biết đâu đấy',
 'biết được',
 'buổi',
 'buổi làm',
 'buổi mới',
 'buổi ngày',
 'buổi sớm',
 'bà',
 'bà ấy',
 'bài',
 'bài bác',
 'bài bỏ',
 'bài cái',
 'bác',
 'bán',
 'bán cấp',
 'bán dạ',
 'bán thế',
 'bây bẩy',
 'bây chừ',
 'bây giờ',
 'bây nhiêu',
 'bèn',
 'béng',
 'bên',
 'bên bị',
 'bên có',
 'bên cạnh',
 'bông',
 'bước',
 'bước khỏi',
 'bước tới',
 'bước đi',
 'bạn',
 'bản',
 'bản bộ',
 'bản riêng',
 'bản thân',
 'bản ý',
 'bất chợt',
 'bất cứ',
 'bất giác',
 'bất kì',
 'bất kể',
 'bất kỳ',
 'bất luận',
 'bất ngờ',
 'bất nhược',
 'bất quá',
 'bất quá chỉ',
 'bất thình l

In [3]:
# import pdfplumber

# def plumber_read(pdffile):
#     content = []
#     with pdfplumber.open(pdffile) as pdf:
#         for page in pdf.pages:
#             content.append(page.extract_text())
            
#     return content


# text = plumber_read("../Hoàng Minh Sơn- BB nghiệm thu CTV-T9.2023.pdf")


# text[4]

In [4]:

from googletrans import Translator

def vi_to_en(text):
  translator = Translator()
  translation = translator.translate(text, src='vi', dest='en')
  return translation.text

def en_to_vi(text):
  translator = Translator()
  translation = translator.translate(text, src='en', dest='vi')
  return translation.text




In [5]:
# text from wikipedia about Elon Musk
content = """
Hướng dẫn cách sử dụng máy chiếu Bước 1: Kết nối dây tín hiệu Để thực hiện bước kết nối dây tín hiệu với máy chiếu. Đầu tiên, bạn phải sử dụng cáp VGA (2 đầu giống nhau) để kết nối, cắm vào cổng có ký hiệu VGA trên cả máy tính và máy chiếu. Lưu ý: Khi cắm cầm phần đầu cắm đẩy mạnh vào khe cắm, vặn chặt đầu cắm để đảm bảo cho khả năng kết nối tốt nhất. Khi tháo bạn cầm phần đầu cắm (không cầm dây) để kéo ra, không bẻ lên bẻ xuống phần dây cắm. Bước 2: Kết nối nguồn điện Nguồn điện để kết nối với máy chiếu, người dùng cần lưu ý về mức điện áp từ 100 - 200V để đảm bảo cho thiết bị máy chiếu hoạt động tốt và ổn định nhất. Lưu ý: Trong khi sử dụng máy chiếu, bạn không nên tắt đột ngột, điều này khiến cho bóng đèn máy chiếu dễ bị hỏng và giảm tuổi thọ chiếu sáng của bóng đèn, cần thực hiện tắt mở máy đúng theo hướng dẫn sử dụng của máy để đảm bảo cho thiết bị hoạt động ổn định nhất. Bước 3: Khởi động máy chiếu Khi khởi động máy chiếu, thực hiện như sau: Mở nắp che đèn chiếu (nếu có) >> Bật công tắc phía sau thân máy (nếu có) >> Nhấn nút Power (1 lần). Trong trường hợp máy chiếu vừa tắt, để mở lại bạn chờ cho quạt trong máy ngừng quay mới bật lại. Bước 4: Xuất hình ra máy chiếu Khi máy chiếu và máy tính đã thực hiện các bước kết nối và khởi động xong thì bước tiếp theo là bạn chỉ cần xuất hình ra máy chiếu để trình chiếu. Tuy nhiên, nếu tín hiệu vẫn chưa xuất ra, bạn kiểm tra: - Kiểm tra Cable nối và máy chiếu: Chọn đúng cổng suất tín hiệu - Kiểm tra máy tính (hoặc các nguồn tín hiệu khác) mở cổng tín hiệu Hoặc bạn cũng có thể tiến hành kiểm tra như sau: - Click chuột phải tại Desktop/Graphics Option/Output to/Desktop - Hoặc kết nối và bật máy chiếu trước khi bật Laptop Bước 5: Sử dụng các nút điều khiển trên máy chiếu Tùy vào vị trí trình chiếu, khoảng cách giữa máy và màn chiếu xa gần mà chúng ta có thể dùng các nút điều khiển để sử dụng máy chiếu. Một số nút lệnh điều khiển thông thường trên máy chiếu như sau: - Điều chỉnh nút TILT: Dùng để nâng cao, hạ thấp độ cao của đèn - Điều chỉnh nút Zoom: để phóng to, thu nhỏ kích thước hình ảnh - Điều chỉnh nút Focus: Để chỉnh độ nét của hình ảnh (Một số dòng dùng Auto Focus). - Các bạn cần kết hợp nút Zoom và Focus để chỉnh đèn sao cho nội dung thấy rõ nhất. Bước 6: Tắt máy chiếu Để tắt máy chiếu sau khi đã sử dụng xong, bằng cách nhấn nút POWER (2 lần) Lưu ý: Bạn nên chờ cho quạt của máy chiếu ngưng hoạt động hẳn thì mới rút dây nguồn ra để tránh nguy cơ hư hỏng và giảm tuổi thọ đèn chiếu.
"""
txt = content

In [6]:
test = "Hướng dẫn cách sử dụng máy chiếu Bước 1: Kết nối dây tín hiệu Để thực hiện bước kết nối dây tín hiệu với máy chiếu. Đầu tiên, bạn phải sử dụng cáp VGA (2 đầu giống nhau) để kết nối, cắm vào cổng có ký hiệu VGA trên cả máy tính và máy chiếu. Lưu ý: Khi cắm cầm phần đầu cắm đẩy mạnh vào khe cắm, vặn chặt đầu cắm để đảm bảo cho khả năng kết nối tốt nhất. Khi tháo bạn cầm phần đầu cắm (không cầm dây) để kéo ra, không bẻ lên bẻ xuống phần dây cắm. Bước 2: Kết nối nguồn điện Nguồn điện để kết nối với máy chiếu, người dùng cần lưu ý về mức điện áp từ 100 - 200V để đảm bảo cho thiết bị máy chiếu hoạt động tốt và ổn định nhất. Lưu ý: Trong khi sử dụng máy chiếu, bạn không nên tắt đột ngột, điều này khiến cho bóng đèn máy chiếu dễ bị hỏng và giảm tuổi thọ chiếu sáng của bóng đèn, cần thực hiện tắt mở máy đúng theo hướng dẫn sử dụng của máy để đảm bảo cho thiết bị hoạt động ổn định nhất. Bước 3: Khởi động máy chiếu Khi khởi động máy chiếu, thực hiện như sau: Mở nắp che đèn chiếu (nếu có) >> Bật công tắc phía sau thân máy (nếu có) >> Nhấn nút Power (1 lần). Trong trường hợp máy chiếu vừa tắt, để mở lại bạn chờ cho quạt trong máy ngừng quay mới bật lại. Bước 4: Xuất hình ra máy chiếu Khi máy chiếu và máy tính đã thực hiện các bước kết nối và khởi động xong thì bước tiếp theo là bạn chỉ cần xuất hình ra máy chiếu để trình chiếu. Tuy nhiên, nếu tín hiệu vẫn chưa xuất ra, bạn kiểm tra: - Kiểm tra Cable nối và máy chiếu: Chọn đúng cổng suất tín hiệu - Kiểm tra máy tính (hoặc các nguồn tín hiệu khác) mở cổng tín hiệu Hoặc bạn cũng có thể tiến hành kiểm tra như sau: - Click chuột phải tại Desktop/Graphics Option/Output to/Desktop - Hoặc kết nối và bật máy chiếu trước khi bật Laptop Bước 5: Sử dụng các nút điều khiển trên máy chiếu Tùy vào vị trí trình chiếu, khoảng cách giữa máy và màn chiếu xa gần mà chúng ta có thể dùng các nút điều khiển để sử dụng máy chiếu. Một số nút lệnh điều khiển thông thường trên máy chiếu như sau: - Điều chỉnh nút TILT: Dùng để nâng cao, hạ thấp độ cao của đèn - Điều chỉnh nút Zoom: để phóng to, thu nhỏ kích thước hình ảnh - Điều chỉnh nút Focus: Để chỉnh độ nét của hình ảnh (Một số dòng dùng Auto Focus). - Các bạn cần kết hợp nút Zoom và Focus để chỉnh đèn sao cho nội dung thấy rõ nhất. Bước 6: Tắt máy chiếu Để tắt máy chiếu sau khi đã sử dụng xong, bằng cách nhấn nút POWER (2 lần) Lưu ý: Bạn nên chờ cho quạt của máy chiếu ngưng hoạt động hẳn thì mới rút dây nguồn ra để tránh nguy cơ hư hỏng và giảm tuổi thọ đèn chiếu."

tokens = underthesea.sent_tokenize(test)

tokens

['Hướng dẫn cách sử dụng máy chiếu Bước 1: Kết nối dây tín hiệu Để thực hiện bước kết nối dây tín hiệu với máy chiếu.',
 'Đầu tiên, bạn phải sử dụng cáp VGA (2 đầu giống nhau) để kết nối, cắm vào cổng có ký hiệu VGA trên cả máy tính và máy chiếu.',
 'Lưu ý: Khi cắm cầm phần đầu cắm đẩy mạnh vào khe cắm, vặn chặt đầu cắm để đảm bảo cho khả năng kết nối tốt nhất.',
 'Khi tháo bạn cầm phần đầu cắm (không cầm dây) để kéo ra, không bẻ lên bẻ xuống phần dây cắm.',
 'Bước 2: Kết nối nguồn điện Nguồn điện để kết nối với máy chiếu, người dùng cần lưu ý về mức điện áp từ 100 - 200V để đảm bảo cho thiết bị máy chiếu hoạt động tốt và ổn định nhất.',
 'Lưu ý: Trong khi sử dụng máy chiếu, bạn không nên tắt đột ngột, điều này khiến cho bóng đèn máy chiếu dễ bị hỏng và giảm tuổi thọ chiếu sáng của bóng đèn, cần thực hiện tắt mở máy đúng theo hướng dẫn sử dụng của máy để đảm bảo cho thiết bị hoạt động ổn định nhất.',
 'Bước 3: Khởi động máy chiếu Khi khởi động máy chiếu, thực hiện như sau: Mở nắp che đ

In [7]:
#test = "Elon Reeve Musk FRS (/ˈiːlɒn/ EE-lon; born June 28, 1971) is an entrepreneur and business magnate. He is the founder, CEO, and Chief Engineer at SpaceX; early stage investor,[note 1] CEO, and Product Architect of Tesla, Inc.; founder of The Boring Company; and co-founder of Neuralink and OpenAI. A centibillionaire, Musk is one of the richest people in the world.Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University but decided instead to pursue a business career, co-founding the web software company Zip2 with his brother Kimbal. The startup was acquired by Compaq for $307 million in 1999. Musk co-founded online bank X.com that same year, which merged with Confinity in 2000 to form PayPal. The company was bought by eBay in 2002 for $1.5 billion.In 2002, Musk founded SpaceX, an aerospace manufacturer and space transport services company, of which he is CEO and CTO. In 2004, he joined electric vehicle manufacturer Tesla Motors, Inc. (now Tesla, Inc.) as chairman and product architect, becoming its CEO in 2008. In 2006, he helped create SolarCity, a solar energy services company that was later acquired by Tesla and became Tesla Energy. In 2015, he co-founded OpenAI, a nonprofit research company that promotes friendly artificial intelligence. In 2016, he co-founded Neuralink, a neurotechnology company focused on developing brain–computer interfaces, and founded The Boring Company, a tunnel construction company. Musk has proposed the Hyperloop, a high-speed vactrain transportation system.Musk has been the subject of criticism due to unorthodox or unscientific stances and highly publicized controversies. In 2018, he was sued for defamation by a diver who advised in the Tham Luang cave rescue; a California jury ruled in favor of Musk. In the same year, he was sued by the US Securities and Exchange Commission (SEC) for falsely tweeting that he had secured funding for a private takeover of Tesla. He settled with the SEC, temporarily stepping down from his chairmanship and accepting limitations on his Twitter usage. Musk has spread misinformation about the COVID-19 pandemic and has received criticism from experts for his other views on such matters as artificial intelligence and public transport."
test = "Hướng dẫn cách sử dụng máy chiếu Bước 1: Kết nối dây tín hiệu Để thực hiện bước kết nối dây tín hiệu với máy chiếu. Đầu tiên, bạn phải sử dụng cáp VGA (2 đầu giống nhau) để kết nối, cắm vào cổng có ký hiệu VGA trên cả máy tính và máy chiếu. Lưu ý: Khi cắm cầm phần đầu cắm đẩy mạnh vào khe cắm, vặn chặt đầu cắm để đảm bảo cho khả năng kết nối tốt nhất. Khi tháo bạn cầm phần đầu cắm (không cầm dây) để kéo ra, không bẻ lên bẻ xuống phần dây cắm. Bước 2: Kết nối nguồn điện Nguồn điện để kết nối với máy chiếu, người dùng cần lưu ý về mức điện áp từ 100 - 200V để đảm bảo cho thiết bị máy chiếu hoạt động tốt và ổn định nhất. Lưu ý: Trong khi sử dụng máy chiếu, bạn không nên tắt đột ngột, điều này khiến cho bóng đèn máy chiếu dễ bị hỏng và giảm tuổi thọ chiếu sáng của bóng đèn, cần thực hiện tắt mở máy đúng theo hướng dẫn sử dụng của máy để đảm bảo cho thiết bị hoạt động ổn định nhất. Bước 3: Khởi động máy chiếu Khi khởi động máy chiếu, thực hiện như sau: Mở nắp che đèn chiếu (nếu có) >> Bật công tắc phía sau thân máy (nếu có) >> Nhấn nút Power (1 lần). Trong trường hợp máy chiếu vừa tắt, để mở lại bạn chờ cho quạt trong máy ngừng quay mới bật lại. Bước 4: Xuất hình ra máy chiếu Khi máy chiếu và máy tính đã thực hiện các bước kết nối và khởi động xong thì bước tiếp theo là bạn chỉ cần xuất hình ra máy chiếu để trình chiếu. Tuy nhiên, nếu tín hiệu vẫn chưa xuất ra, bạn kiểm tra: - Kiểm tra Cable nối và máy chiếu: Chọn đúng cổng suất tín hiệu - Kiểm tra máy tính (hoặc các nguồn tín hiệu khác) mở cổng tín hiệu Hoặc bạn cũng có thể tiến hành kiểm tra như sau: - Click chuột phải tại Desktop/Graphics Option/Output to/Desktop - Hoặc kết nối và bật máy chiếu trước khi bật Laptop Bước 5: Sử dụng các nút điều khiển trên máy chiếu Tùy vào vị trí trình chiếu, khoảng cách giữa máy và màn chiếu xa gần mà chúng ta có thể dùng các nút điều khiển để sử dụng máy chiếu. Một số nút lệnh điều khiển thông thường trên máy chiếu như sau: - Điều chỉnh nút TILT: Dùng để nâng cao, hạ thấp độ cao của đèn - Điều chỉnh nút Zoom: để phóng to, thu nhỏ kích thước hình ảnh - Điều chỉnh nút Focus: Để chỉnh độ nét của hình ảnh (Một số dòng dùng Auto Focus). - Các bạn cần kết hợp nút Zoom và Focus để chỉnh đèn sao cho nội dung thấy rõ nhất. Bước 6: Tắt máy chiếu Để tắt máy chiếu sau khi đã sử dụng xong, bằng cách nhấn nút POWER (2 lần) Lưu ý: Bạn nên chờ cho quạt của máy chiếu ngưng hoạt động hẳn thì mới rút dây nguồn ra để tránh nguy cơ hư hỏng và giảm tuổi thọ đèn chiếu."

tokens = nltk.sent_tokenize(test)

def clean_sentence(sentence, stopwords=False):
    sentence = sentence.lower().strip()
    # sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    sentence = re.sub(r'[^a-zA-Z0-9\sÀÁẢẠÃĂẰẮẲẶẴÂẦẤẨẬẪÈÉẺẸẼÊỀẾỂỆỄÌÍỈỊĨÒÓỎỌÕÔỒỐỔỘỖƠỜỚỞỢỠÙÚỦỤŨƯỪỨỬỰỮỲÝỶỴỸàáảạãăằắẳặẵâầấẩậẫèéẻẹẽêềếểệễìíỉịĩòóỏọõôồốổộỗơờớởợỡùúủụũưừứửựữỳýỷỵỹđ]+', '', sentence)
    if stopwords:
        sentence = remove_stopwords(sentence ,stopwords=vi_stopwords)
    return sentence

cleaned_sentences = []
for line in tokens:
    cleaned = clean_sentence(line, True)
    cleaned_sentences.append(cleaned)
    
print(tokens)
print(cleaned_sentences)

['Hướng dẫn cách sử dụng máy chiếu Bước 1: Kết nối dây tín hiệu Để thực hiện bước kết nối dây tín hiệu với máy chiếu.', 'Đầu tiên, bạn phải sử dụng cáp VGA (2 đầu giống nhau) để kết nối, cắm vào cổng có ký hiệu VGA trên cả máy tính và máy chiếu.', 'Lưu ý: Khi cắm cầm phần đầu cắm đẩy mạnh vào khe cắm, vặn chặt đầu cắm để đảm bảo cho khả năng kết nối tốt nhất.', 'Khi tháo bạn cầm phần đầu cắm (không cầm dây) để kéo ra, không bẻ lên bẻ xuống phần dây cắm.', 'Bước 2: Kết nối nguồn điện Nguồn điện để kết nối với máy chiếu, người dùng cần lưu ý về mức điện áp từ 100 - 200V để đảm bảo cho thiết bị máy chiếu hoạt động tốt và ổn định nhất.', 'Lưu ý: Trong khi sử dụng máy chiếu, bạn không nên tắt đột ngột, điều này khiến cho bóng đèn máy chiếu dễ bị hỏng và giảm tuổi thọ chiếu sáng của bóng đèn, cần thực hiện tắt mở máy đúng theo hướng dẫn sử dụng của máy để đảm bảo cho thiết bị hoạt động ổn định nhất.', 'Bước 3: Khởi động máy chiếu Khi khởi động máy chiếu, thực hiện như sau: Mở nắp che đèn chi

In [8]:
#class for preprocessing and creating word embedding
class Preprocessing:
    #constructor
    def __init__(self,txt):
        # Tokenization
        nltk.download('punkt')  #punkt is nltk tokenizer 
        # breaking text to sentences
        tokens = nltk.sent_tokenize(txt) 
        self.tokens = tokens
        self.tfidfvectoriser=TfidfVectorizer()

    # Data Cleaning
    # remove extra spaces
    # convert sentences to lower case 
    # remove stopword
    def clean_sentence(self, sentence, stopwords=False):
        sentence = sentence.lower().strip()
        sentence = re.sub(r'[^a-zA-Z0-9\sÀÁẢẠÃĂẰẮẲẶẴÂẦẤẨẬẪÈÉẺẸẼÊỀẾỂỆỄÌÍỈỊĨÒÓỎỌÕÔỒỐỔỘỖƠỜỚỞỢỠÙÚỦỤŨƯỪỨỬỰỮỲÝỶỴỸàáảạãăằắẳặẵâầấẩậẫèéẻẹẽêềếểệễìíỉịĩòóỏọõôồốổộỗơờớởợỡùúủụũưừứửựữỳýỷỵỹđ]+', '', sentence)
        if stopwords:
            sentence = remove_stopwords(sentence ,stopwords=vi_stopwords)
        return sentence

    # store cleaned sentences to cleaned_sentences
    def get_cleaned_sentences(self,tokens, stopwords=False):
        cleaned_sentences = []
        for line in tokens:
            cleaned = self.clean_sentence(line, stopwords)
            cleaned_sentences.append(cleaned)
        return cleaned_sentences

    #do all the cleaning
    def cleanall(self):
        cleaned_sentences = self.get_cleaned_sentences(self.tokens, stopwords=True)
        cleaned_sentences_with_stopwords = self.get_cleaned_sentences(self.tokens, stopwords=False)
        return [cleaned_sentences,cleaned_sentences_with_stopwords]

    # TF-IDF Vectorizer
    def TFIDF(self,cleaned_sentences):
        self.tfidfvectoriser.fit(cleaned_sentences)
        tfidf_vectors=self.tfidfvectoriser.transform(cleaned_sentences)
        return tfidf_vectors

    #tfidf for question
    def TFIDF_Q(self,question_to_be_cleaned):
        tfidf_vectors=self.tfidfvectoriser.transform([question_to_be_cleaned])
        return tfidf_vectors

    # main call function
    def doall(self):
        cleaned_sentences, cleaned_sentences_with_stopwords = self.cleanall()
        tfidf = self.TFIDF(cleaned_sentences)
        return [cleaned_sentences,cleaned_sentences_with_stopwords,tfidf]

In [9]:
#class for answering the question.
class AnswerMe:
    #cosine similarity
    def Cosine(self, question_vector, sentence_vector):
        dot_product = np.dot(question_vector, sentence_vector.T)
        denominator = (np.linalg.norm(question_vector) * np.linalg.norm(sentence_vector))
        return dot_product/denominator
    
    #Euclidean distance
    def Euclidean(self, question_vector, sentence_vector):
        vec1 = question_vector.copy()
        vec2 = sentence_vector.copy()
        if len(vec1)<len(vec2): vec1,vec2 = vec2,vec1
        vec2 = np.resize(vec2,(vec1.shape[0],vec1.shape[1]))
        return np.linalg.norm(vec1-vec2)

    # main call function
    def answer(self, question_vector, sentence_vector, method):
        if method==1: return self.Euclidean(question_vector,sentence_vector)
        else: return self.Cosine(question_vector,sentence_vector)

In [10]:

def RetrieveAnswer(question_embedding, tfidf_vectors,method=1):
    similarity_heap = []
    if method==1: max_similarity = float('inf')
    else: max_similarity = -1
    index_similarity = -1

    for index, embedding in enumerate(tfidf_vectors):  
        find_similarity = AnswerMe()
        similarity = find_similarity.answer((question_embedding).toarray(),(embedding).toarray() , method).mean()
        print("SIMI", similarity)
        if method==1:
            heapq.heappush(similarity_heap,(similarity,index))
        else:
            heapq.heappush(similarity_heap,(-similarity,index))
            
    return similarity_heap

In [11]:
# Put Your question here
user_question = "Đưa cho tôi hướng dẫn đầy đủ sử dụng máy chiếu"
#define method
method = 1

In [12]:
preprocess = Preprocessing(txt)
cleaned_sentences,cleaned_sentences_with_stopwords,tfidf_vectors = preprocess.doall()

question = preprocess.clean_sentence(user_question, stopwords=True)
question_embedding = preprocess.TFIDF_Q(question)

similarity_heap = RetrieveAnswer(question_embedding , tfidf_vectors ,method)

SIMI 1.0619466061641178
SIMI 1.2156503037627664
SIMI 1.414213562373095
SIMI 1.4142135623730951
SIMI 1.3314498163482664
SIMI 0.9868712833992181
SIMI 1.272262283251299
SIMI 1.3002530107420112
SIMI 1.222918865491812
SIMI 1.2108921484824091
SIMI 1.387395656000638
SIMI 1.4142135623730951
SIMI 1.1613343614211873


[nltk_data] Downloading package punkt to /home/tuannm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
print("Question: ", user_question)

# number of relevant solutions you want here it will print 2
number_of_sentences_to_print = 50

while number_of_sentences_to_print>0 and len(similarity_heap)>0:
    x = similarity_heap.pop(0)
    print(x[0])
    print(cleaned_sentences_with_stopwords[x[1]])
    print("\n")
    number_of_sentences_to_print-=1

Question:  Đưa cho tôi hướng dẫn đầy đủ sử dụng máy chiếu
0.9868712833992181
lưu ý trong khi sử dụng máy chiếu bạn không nên tắt đột ngột điều này khiến cho bóng đèn máy chiếu dễ bị hỏng và giảm tuổi thọ chiếu sáng của bóng đèn cần thực hiện tắt mở máy đúng theo hướng dẫn sử dụng của máy để đảm bảo cho thiết bị hoạt động ổn định nhất


1.2108921484824091
tuy nhiên nếu tín hiệu vẫn chưa xuất ra bạn kiểm tra  kiểm tra cable nối và máy chiếu chọn đúng cổng suất tín hiệu  kiểm tra máy tính hoặc các nguồn tín hiệu khác mở cổng tín hiệu hoặc bạn cũng có thể tiến hành kiểm tra như sau  click chuột phải tại desktopgraphics optionoutput todesktop  hoặc kết nối và bật máy chiếu trước khi bật laptop bước 5 sử dụng các nút điều khiển trên máy chiếu tùy vào vị trí trình chiếu khoảng cách giữa máy và màn chiếu xa gần mà chúng ta có thể dùng các nút điều khiển để sử dụng máy chiếu


1.0619466061641178
hướng dẫn cách sử dụng máy chiếu bước 1 kết nối dây tín hiệu để thực hiện bước kết nối dây tín hiệu 